# 🚀 Singularity V3 - AI Model Training

Developed by Molty for Bally. This notebook implements the **V3 Architecture**: 
- **Temporal CNN** for local feature extraction
- **Bi-LSTM** for sequence dependency
- **Multi-Head Attention** for global context
- **Huber Loss** for robust regression

In [ ]:
# 1. Setup Environment
!pip install torch pandas numpy scikit-learn matplotlib

# --- NEW: Merge Data Chunks from GitHub ---
!cat dataset.csv.part* > dataset.csv
print("✅ Dataset merged from chunks.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# --- Hyperparameters ---
SEQ_LENGTH = 60
BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 1e-4
FEATURE_COLS = ['velocity', 'accel', 'entropy', 'mass', 'imbalance', 'liq_force']
TARGET_COL = 'label_return_60s'

## 2. Upload Data
Please upload your `dataset.csv` using the folder icon on the left.

In [ ]:
# --- 1. 數據抓取與合併 ---
if not os.path.exists('dataset.csv'):
    print("🚀 正在從 GitHub 抓取數據塊...")
    !wget -q https://raw.githubusercontent.com/bally65/singularity/master/dataset.csv.partaa
    !wget -q https://raw.githubusercontent.com/bally65/singularity/master/dataset.csv.partab
    !wget -q https://raw.githubusercontent.com/bally65/singularity/master/dataset.csv.partac
    !wget -q https://raw.githubusercontent.com/bally65/singularity/master/dataset.csv.partad
    print("📂 正在合併數據塊...")
    !cat dataset.csv.part* > dataset.csv
else:
    print("📦 數據檔案已存在，跳過抓取。")

# --- 2. 數據讀取與清理 ---
print("📊 正在讀取數據 (自動修復接縫錯誤)...")
try:
    # 使用 on_bad_lines='skip' 處理分割產生的殘缺行
    df = pd.read_csv('dataset.csv', on_bad_lines='skip', low_memory=False).dropna()
    print(f"✅ 成功讀取 {len(df)} 筆有效數據！")
except Exception as e:
    print(f"❌ 讀取失敗: {e}")

## 3. Define Model V3

In [ ]:
class FinancialDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
    def __len__(self): return len(self.features)
    def __getitem__(self, idx): return self.features[idx], self.labels[idx]

class SingularityV3Model(nn.Module):
    def __init__(self, input_dim, d_model=128, nhead=8):
        super().__init__()
        self.conv1 = nn.Conv1d(input_dim, d_model, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(d_model, d_model, num_layers=2, batch_first=True, dropout=0.1)
        self.attn = nn.MultiheadAttention(d_model, nhead, dropout=0.1)
        self.head = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, 3) # [Q10, Q50, Q90]
        )

    def forward(self, x):
        x = x.transpose(1, 2)
        x = F.relu(self.conv1(x))
        x = x.transpose(1, 2)
        lstm_out, _ = self.lstm(x)
        attn_in = lstm_out.transpose(0, 1)
        attn_out, _ = self.attn(attn_in, attn_in, attn_in)
        last_state = attn_out[-1]
        return self.head(last_state)

## 4. Train

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, shuffle=False)
train_loader = DataLoader(FinancialDataset(X_train, Y_train), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(FinancialDataset(X_val, Y_val), batch_size=BATCH_SIZE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SingularityV3Model(len(FEATURE_COLS)).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.HuberLoss()

train_losses = []
print(f"🚀 Training V3 on {device}...")
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)[:, 1] # Target median
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss/len(train_loader)
    train_losses.append(avg_loss)
    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.6f}")

plt.plot(train_losses)
plt.title("V3 Training Loss")
plt.show()

## 5. Export to ONNX
This will create `singularity_v3.onnx` for the Go Engine.

In [ ]:
model.eval()
dummy = torch.randn(1, SEQ_LENGTH, len(FEATURE_COLS)).to(device)
torch.onnx.export(model, dummy, "singularity_v3.onnx", input_names=['input'], output_names=['output'])
print("✅ Done! Download singularity_v3.onnx and put it in your Go project root.")